In [1]:
import sys
sys.path.append('..')
from utils import load_config
class Arguments: pass
args=Arguments()
args.config = '../configs/config.yaml'
args.config_workspace = '../configs/config_aia.yaml'
cfg=load_config(args)

Config path 
 config: ../configs/config.yaml 
 config_workspace: ../configs/config_aia.yaml


In [26]:
import segmentation_models_pytorch as smp
import torch.nn as nn
import torch

In [3]:
encoder_name=cfg["smp"]["encoder_name"]
num_cls=cfg["num_cls"]
# https://smp.readthedocs.io/en/latest/_modules/segmentation_models_pytorch/decoders/unetplusplus/model.html#UnetPlusPlus
model = smp.UnetPlusPlus(encoder_name=encoder_name,
                         encoder_depth=5,
                         encoder_weights='imagenet',
                         decoder_use_batchnorm=True,
                         decoder_channels=(256, 128, 64, 32, 16),
                         decoder_attention_type=None,
                         in_channels=3,
                         classes=num_cls, activation=None, aux_params=None)

In [6]:
model = smp.UnetPlusPlus(encoder_name=encoder_name,
                         encoder_depth=5,
                         encoder_weights='imagenet',
                         decoder_use_batchnorm=True,
                         decoder_channels=(256, 128, 64, 32, 16),
                         decoder_attention_type=None,
                         in_channels=3,
                         classes=num_cls, activation=None, aux_params=None)

In [ ]:
nn.LayerNorm()

In [5]:
a=dict(a=1,b=2,c=3,d=4)

In [17]:
{k:v for k,v in a.items() if k in ['b','c'] }

{'b': 2, 'c': 3}

In [8]:
a

{'a': 1, 'b': 2, 'c': 3, 'd': 4}

In [9]:
model.encoder.cuda()

EfficientNetEncoder(
  (conv_stem): Conv2d(3, 48, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(48, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (act1): Swish()
  (blocks): Sequential(
    (0): Sequential(
      (0): DepthwiseSeparableConv(
        (conv_dw): Conv2d(48, 48, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=48, bias=False)
        (bn1): BatchNorm2d(48, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (act1): Swish()
        (se): SqueezeExcite(
          (conv_reduce): Conv2d(48, 12, kernel_size=(1, 1), stride=(1, 1))
          (act1): Swish()
          (conv_expand): Conv2d(12, 48, kernel_size=(1, 1), stride=(1, 1))
          (gate): Sigmoid()
        )
        (conv_pw): Conv2d(48, 24, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn2): BatchNorm2d(24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (act2): Identity()
      )
      (1): Depthwis

In [21]:
next(model.parameters()).device

device(type='cuda', index=0)

In [5]:
import torch


In [25]:
torch.device("cuda")

device(type='cuda')

In [6]:
ntype={"bn":nn.BatchNorm2d,
       "in":nn.InstanceNorm2d
      }

In [7]:
import re
def get_hyp_params(module):
    args=[]
    kwargs={}

    repr_=module.__repr__()
    print(repr_)
    for string in re.search(r"(\()(.+)(\))",repr_)[2].split(","):
        if "=" in string:
            sep=string.strip(" ").split("=")
            kwargs[sep[0]]=eval(sep[1])
        else:
            args+=[eval(string)]
    return args,kwargs

In [8]:
hyp=get_hyp_params(model.encoder.bn1)
hyp

BatchNorm2d(48, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)


([48],
 {'eps': 1e-05, 'momentum': 0.1, 'affine': True, 'track_running_stats': True})

In [10]:
ntype["in"](*hyp[0],**hyp[1])

InstanceNorm2d(48, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)

In [241]:
for mod in model.children():
    if type(mod) in ntype.values():
        print(mod.__repr__())

In [14]:
target_type="in"
other_types=[v for k,v in ntype.items() if k!=target_type]
for n,c in model.encoder.named_children():
    if type(c) in ntype.values():
        if type(c) in other_types:
            hyp=get_hyp_params(c)
            _=model.encoder.__setattr__(n,ntype[target_type](*hyp[0],**hyp[1]))
            

In [ ]:
model

In [18]:
ntype={"bn":nn.BatchNorm2d,
       "in":nn.InstanceNorm2d
      }
target_type="in"
other_types=[v for k,v in ntype.items() if k!=target_type]

def change_all(sub):
    for n,c in sub.named_children():
        if type(c) in ntype.values():
            if type(c) in other_types:
                hyp=get_hyp_params(c)
                _=sub.__setattr__(n,ntype[target_type](*hyp[0],**hyp[1]))
        change_all(c)

In [19]:
change_all(model)

In [20]:
model

UnetPlusPlus(
  (encoder): EfficientNetEncoder(
    (conv_stem): Conv2d(3, 48, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (bn1): InstanceNorm2d(48, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (act1): Swish()
    (blocks): Sequential(
      (0): Sequential(
        (0): DepthwiseSeparableConv(
          (conv_dw): Conv2d(48, 48, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=48, bias=False)
          (bn1): InstanceNorm2d(48, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (act1): Swish()
          (se): SqueezeExcite(
            (conv_reduce): Conv2d(48, 12, kernel_size=(1, 1), stride=(1, 1))
            (act1): Swish()
            (conv_expand): Conv2d(12, 48, kernel_size=(1, 1), stride=(1, 1))
            (gate): Sigmoid()
          )
          (conv_pw): Conv2d(48, 24, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn2): InstanceNorm2d(24, eps=1e-05, momentum=0.1, affine=True, track_runn

In [16]:
from models.action import CNNLSTM,LeNetVariant
from models.head import MetricLayer

In [18]:
LeNetVariant

models.action.LeNetVariant

In [19]:
model=LeNetVariant().cuda()#encoder_cout=128,num_classes=8).cuda()

In [20]:
classifier_name,classifier=[*model.named_children()][-1]

In [21]:
import re
def get_hyp_params(module):
    args=[]
    kwargs={}
    repr_=module.__repr__()
    for string in re.search(r"(\()(.+)(\))",repr_)[2].split(","):
        if "=" in string:
            sep=string.strip(" ").split("=")
            kwargs[sep[0]]=eval(sep[1])
        else:
            args+=[eval(string)]
    return args,kwargs

In [22]:
args,kwargs=get_hyp_params(classifier)

In [23]:
new_classifier=MetricLayer(n_in_features=kwargs["in_features"],n_out_features=kwargs["out_features"])

KeyError: 'in_features'

In [1]:
from collections import defaultdict

In [4]:
a=defaultdict(lambda:0)

In [16]:
[*new_classifier.parameters()][0]

Parameter containing:
tensor([[-0.1673, -0.2050, -0.0889,  ...,  0.1820,  0.0331,  0.1033],
        [-0.2019,  0.1783, -0.1026,  ...,  0.0321, -0.1477,  0.0468],
        [-0.0197, -0.1717, -0.1044,  ..., -0.2026,  0.0525,  0.0229],
        ...,
        [ 0.1939,  0.1089, -0.0595,  ...,  0.1800,  0.0369,  0.1700],
        [-0.0494, -0.0086, -0.0345,  ..., -0.1888, -0.0214, -0.0404],
        [-0.0578,  0.0989,  0.1340,  ...,  0.0913,  0.0275,  0.1141]],
       device='cuda:0', requires_grad=True)

In [19]:
_=model.__setattr__(classifier_name,new_classifier)

In [17]:
classifier

Linear(in_features=128, out_features=8, bias=True)

In [20]:
del classifier

In [27]:
inputs = torch.normal(0, 1, (1, 3, 128, 128)).cuda()

In [32]:
inputs.shape

torch.Size([1, 3, 128, 128])

In [34]:
outputs= model(inputs)

RuntimeError: Given groups=1, weight of size [16, 1, 5, 5], expected input[1, 3, 128, 128] to have 1 channels, but got 3 channels instead